In [1]:
#This version of the evolutionary algorithm seeks to implement
#step 5, i.e. concurrency, 
#looks at different objective functions,
#looks at different research scenarios and
#tests actions 2 and 3

In [ ]:
#Initalize an array of tuples, indicating how much production went into them and their type
#Keep track of the amount of construction done by all civs
#Keep track of how many available civs there are, give each entry a max of 15 civs worth of production
#At the start of a new state:
#If there is no production in a slot change it to the factory the model wants to build
#If a factory is >5% finished continue building it
#Production overflow is saved and added to the first unfinished factory in the array afterwards

#NOTE: CONCURRENCY SEEMS TO REQUIRE DAILY UPDATING TO FUNCTION PROPERLY
#EFFECTS OF CONCURRENCY SHOULD BE MINIMAL, ALL INFRASTRUCTURE IS THE SAME FOR EVERY FACTORY
#ATTENTION BETTER FOCUSED ELSEWHERE

In [2]:
#Production efficiency techs take 150 days base, construction and industry take 200 base

#Meta strat:
#21 march 1936: Improved worker conditions
#2 september 1936: War eco
#27 november 1936: Popular figurehead
#30 march 1937: Captain of industry, state 12


#Normal strat:
#17 july 1936: Spanish civil war, war eco


#Research:
#23 may 1936: Basic machine tools, state 4
#8 july 1936: Construction 1, state 5
#24 november 1936: Concentrated 1, state 9 
#28 feburari 1937: Construction 2, state 12
#11 april 1937: Improved machine tools, state 13
#25 may 1937: Concentrated 2, state 14

#ASSUMPTION FOR NOW: All further tech is unlocked 3 months after they become current TEST IN GAME
#Concentrated 3, Machine tools 3, Construction 3, state 33
#Concentrated 4, Machine tools 4, Construction 4, state 54

In [3]:
#STABILITY EVENTS:
#State 2: +5%
#State 4: +5%
#State 8: +10%

In [177]:
import numpy as np
import random


In [91]:
def selection(pop, scores, k=3):
    selection_ix = np.random.randint(len(pop))
    for ix in np.random.randint(0, len(pop), k-1):
        if scores[ix] > scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

def crossover(p1, p2, r_cross):
    c1, c2 = p1.copy(), p2.copy()
    if np.random.rand() < r_cross:
        pt = np.random.randint(1, len(p1)-1)
        #print(pt)
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
        #visualise what solutions have been tried
    return [c1, c2]

def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        if np.random.rand() < r_mut:
            bitstring[i] = np.random.randint(0, 4)#1 - bitstring[i]
    return bitstring

In [92]:
#mutation([1] * 10 + [0] * 10, 1/4)

In [158]:
def update_consumergoods(state, toaster_percent):
    match state:
        case 5:
            toaster_percent -= 0.15 #WAR ECO        
    return round(toaster_percent, 2)

#Have the next three work with different tech scenarios
#one year, half year and current are the different tech scenarios
def update_constructionspeed(state, civ_output_modifier, scenario):
    match scenario:
        case 'one year':
            match state:
                case 0:
                    civ_output_modifier += 0.1 #export focus
                case 5:
                    civ_output_modifier += 0.1
                case 12: #same as half year because we start in 36
                    civ_output_modifier += 0.1 
                case 31:
                    civ_output_modifier += 0.1
                case 52:
                    civ_output_modifier += 0.1
        case 'half year':
            match state:
                case 0:
                    civ_output_modifier += 0.1 #export focus
                case 5:
                    civ_output_modifier += 0.1
                case 12: 
                    civ_output_modifier += 0.1 
                case 33:
                    civ_output_modifier += 0.1
                case 54:
                    civ_output_modifier += 0.1
        case 'current':
            match state:
                case 0:
                    civ_output_modifier += 0.1 #export focus
                case 5:
                    civ_output_modifier += 0.1
                case 16: 
                    civ_output_modifier += 0.1 
                case 36:
                    civ_output_modifier += 0.1
                case 57:
                    civ_output_modifier += 0.1
            
    return round(civ_output_modifier, 2)

def update_miloutput(state, mil_output_modifier, scenario):
    match scenario:
        case 'one year':
            match state:
                case 9:
                    mil_output_modifier += 0.15
                case 14: 
                    mil_output_modifier += 0.15
                case 31:
                    mil_output_modifier += 0.15
                case 52:
                    mil_output_modifier += 0.15
        case 'half year':
            match state:
                case 9:
                    mil_output_modifier += 0.15
                case 14: 
                    mil_output_modifier += 0.15
                case 33:
                    mil_output_modifier += 0.15
                case 54:
                    mil_output_modifier += 0.15
        case 'current':
            match state:
                case 9:
                    mil_output_modifier += 0.15
                case 16: 
                    mil_output_modifier += 0.15
                case 36:
                    mil_output_modifier += 0.15
                case 57:
                    mil_output_modifier += 0.15

    return round(mil_output_modifier, 2)

def update_prodefficiency(state, production_efficiency_cap, scenario):
    match scenario:
        case 'one year':
            match state:
                case 4:
                    production_efficiency_cap += 0.1
                case 13:
                    production_efficiency_cap += 0.1 
                case 31:
                    production_efficiency_cap += 0.1
                case 52:
                    production_efficiency_cap += 0.1
        case 'half year':
            match state:
                case 4:
                    production_efficiency_cap += 0.1
                case 13: 
                    production_efficiency_cap += 0.1 
                case 33:
                    production_efficiency_cap += 0.1
                case 54:
                    production_efficiency_cap += 0.1
        case 'current':
            match state:
                case 4:
                    production_efficiency_cap += 0.1
                case 15: 
                    production_efficiency_cap += 0.1 
                case 36:
                    production_efficiency_cap += 0.1
                case 57:
                    production_efficiency_cap += 0.1

    return round(production_efficiency_cap, 2)

def update_civcost(state, civ_construction_modifier):
    match state:
        case 0:
            civ_construction_modifier += 0.05 #five year plan
        case 5:
            civ_construction_modifier += 0.3
        case 12: 
            civ_construction_modifier += 0.1 #captain of industry
    return round(civ_construction_modifier, 2)

def update_milcost(state, mil_construction_modifier):
    match state:
        case 5:
            mil_construction_modifier += 0.5
    return round(mil_construction_modifier, 2)

def update_civ_to_milcost(state, civ_to_mil_construction_modifier):
    match state:
        case 5:
            civ_to_mil_construction_modifier += 0.5
    return round(civ_to_mil_construction_modifier, 2)

def update_mil_to_civcost(state, mil_to_civ_construction_modifier):
    match state:
        case 5:
            mil_to_civ_construction_modifier += 0.5
    return round(mil_to_civ_construction_modifier, 2)

def construction_queue_update(construction_queue, action, available_civs, civ_output, state_size_days,
                              infrastructure_modifier, civ_output_modifier, civ_construction_modifier,
                              mil_construction_modifier, civ_to_mil_construction_modifier, 
                             mil_to_civ_construction_modifier):
    free_civs = available_civs
    for i in range(len(construction_queue)):
        if construction_queue[i] == None:
            construction_queue[i] = (0,0)
        
    for x in range(len(construction_queue)):  
        if action == 0 and construction_queue[x][0] == 0:
            construction_queue[x] = (0, 7200, 'military')
        if action == 1 and construction_queue[x][0] == 0:
            construction_queue[x] = (0, 10800, 'civilian')
        if action == 2 and construction_queue[x][0] == 0:
            construction_queue[x] = (0, 4000, 'civ to mil')
        if action == 3 and construction_queue[x][0] == 0:
            construction_queue[x] = (0, 9000, 'mil to civ')
    
    #Calculate how much a single factory would contribute to either of the four actions
    #during one state
    build_civ_output = civ_output * state_size_days * infrastructure_modifier \
    * civ_output_modifier * civ_construction_modifier
    
    build_mil_output = civ_output * state_size_days * infrastructure_modifier \
    * civ_output_modifier * mil_construction_modifier
    
    build_civ_to_mil_output = civ_output * state_size_days * infrastructure_modifier \
    * civ_output_modifier * civ_to_mil_construction_modifier
    
    build_mil_to_civ_output = civ_output * state_size_days * infrastructure_modifier \
    * civ_output_modifier * mil_to_civ_construction_modifier
    
    #print(build_civ_output, build_mil_output, build_civ_to_mil_output, build_mil_to_civ_output)
    for j in range(len(construction_queue)):
        if free_civs > 0 and free_civs < 15:
            print(construction_queue[j][2])
            
            free_civs -= free_civs 
            print(j, free_civs, available_civs)
        if free_civs > 0:
            
            
            free_civs -= 15 
            print(j, free_civs, available_civs)            
        else:
            break
    
    return construction_queue

In [200]:
#Two max_targets: 'rush', 'scale', 'scale_total' and 'scale_civs'

def production(actions, max_target, scenario):
    
    ic_prod_list = []
    mils = 32
    mils_0 = 32 #keep track of the number of starting factories
    civs = 45
    civs_0 = 45 #idem
    total = mils + civs
    total_0 = 32 + 45 #idem
    consumergoods_percentage = 0.40
    consumergoods = np.floor(total * consumergoods_percentage)
    available_civs = np.maximum(int(civs - consumergoods), 0)
    
    #Civ output modifiers are additive (-30% + 15% = -15%), multiplied by state infrastructure
    civ_output = 5 
    civ_output_modifier = 1 #Construction speed
    civ_construction_modifier = 0.7
    mil_construction_modifier = 0.7
    civ_to_mil_construction_modifier = 0.7
    mil_to_civ_construction_modifier = 0.7
    
    mil_output = 4.5
    mil_output_modifier = 1
    prod_eff = 0.5
    production_efficiency_cap = 0.55
    
    #Investigate how new factories add to production efficiency
    #production_efficiency works as on the wiki, except for the starting factories
    #with different rules
    #The number of factories because of trade is quite significant, but ultimately unpredictable
    
    state = 0
    state_size_days = 35 #how many days per state
    ic_produced = 0
    ic_total = 0    
    civ_purchase_pool = 0 
    mil_purchase_pool = 0
    civ_to_mil_pool = 0
    mil_to_civ_pool = 0
    infrastructure_modifier = 1.6
    #construction_queue = np.zeros(30) 
    #construction_queue = np.empty((30), dtype=object) #equates to 450 free civs, more than will be reached in practice
    
    #print(construction_queue_update(construction_queue, 0))
    #construction_queue = construction_queue_update(construction_queue, 1, available_civs, civ_output, state_size_days,
    #                          infrastructure_modifier, civ_output_modifier, civ_construction_modifier,
    #                          mil_construction_modifier, civ_to_mil_construction_modifier, 
    #                         mil_to_civ_construction_modifier)
    #print(construction_queue)
    
    for i in range(len(actions)):
        consumergoods_percentage = (update_consumergoods(i, consumergoods_percentage))
        civ_output_modifier = (update_constructionspeed(i, civ_output_modifier, scenario))
        mil_output_modifier = (update_miloutput(i, mil_output_modifier, scenario))
        civ_construction_modifier = (update_civcost(i, civ_construction_modifier))
        mil_construction_modifier = (update_milcost(i, mil_construction_modifier))
        civ_to_mil_construction_modifier = (update_civ_to_milcost(i, civ_to_mil_construction_modifier))
        mil_to_civ_construction_modifier = (update_mil_to_civcost(i, mil_to_civ_construction_modifier))
        production_efficiency_cap = (update_prodefficiency(i, production_efficiency_cap, scenario))
        
        ic_produced = mil_output * mils * state_size_days * mil_output_modifier * prod_eff
        ic_prod_list.append(mils) #ic_produced
        
        if actions[i] == 0: #build mils     
            mil_purchase_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * mil_construction_modifier)
            
            existing_mils = mils #Used exclusively for production efficiency calculations
            #construction_queue = construction_queue_update(construction_queue, actions[i], available_civs, civ_output, state_size_days,
            #                  infrastructure_modifier, civ_output_modifier, civ_construction_modifier,
            #                  mil_construction_modifier, civ_to_mil_construction_modifier, 
            #                  mil_to_civ_construction_modifier)
            #print(construction_queue)
            
            mils += np.floor_divide(mil_purchase_pool, 7200) 
            total += np.floor_divide(mil_purchase_pool, 7200) 
            consumergoods = np.ceil(total * consumergoods_percentage)
            mil_purchase_pool = np.mod(mil_purchase_pool, 7200)
            available_civs = np.maximum(int(civs - consumergoods), 0)
            
            prod_eff = (existing_mils * prod_eff + (mils - existing_mils) * 0.1) / mils
            growth = 0.001 * (production_efficiency_cap **2 / prod_eff)
            prod_eff += (growth * state_size_days) / 2
            prod_eff = min(1, prod_eff)
            
            state += 1
            ic_total += ic_produced
            
        if actions[i] == 1: #build civs         
            civ_purchase_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * civ_construction_modifier)
            #construction_queue = construction_queue_update(construction_queue, actions[i], available_civs, civ_output, state_size_days,
            #                  infrastructure_modifier, civ_output_modifier, civ_construction_modifier,
            #                  mil_construction_modifier, civ_to_mil_construction_modifier, 
            #                  mil_to_civ_construction_modifier)
            civs += np.floor_divide(civ_purchase_pool, 10800) 
            total += np.floor_divide(civ_purchase_pool, 10800) 
            consumergoods = np.ceil(total * consumergoods_percentage)
            civ_purchase_pool = np.mod(civ_purchase_pool, 10800)
            available_civs = np.maximum(int(civs - consumergoods), 0)
            
            growth = 0.001 * (production_efficiency_cap **2 / prod_eff)
            prod_eff += (growth * state_size_days) / 2
            prod_eff = min(1, prod_eff)
            
            state += 1
            ic_total += ic_produced

        if actions[i] == 2: #Convert civs to mils
            civ_to_mil_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * civ_to_mil_construction_modifier)

            civs -= np.floor_divide(civ_to_mil_pool, 4000)
            mils += np.floor_divide(civ_to_mil_pool, 4000)
            civ_to_mil_pool = np.mod(civ_to_mil_pool, 4000)
            consumergoods = np.ceil(total * consumergoods_percentage)
            available_civs = np.maximum(int(civs - consumergoods), 0)
            ic_total += ic_produced
        
        if actions[i] == 3: #Convert mils to civs
            mil_to_civ_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * mil_to_civ_construction_modifier)
            if mils > 0:
                civs += np.floor_divide(mil_to_civ_pool, 9000)
                mils -= np.floor_divide(mil_to_civ_pool, 9000)
                
                if mils < 0:
                    too_many_removed = abs(mils)
                    civs -= too_many_removed
                    mils = 0
                    mil_to_civ_pool = 0
                
                else:
                    mil_to_civ_pool = np.mod(mil_to_civ_pool, 9000)
                consumergoods = np.ceil(total * consumergoods_percentage)
                available_civs = np.maximum(int(civs - consumergoods), 0)
            ic_total += ic_produced
            
    match max_target:
        case 'rush':
            return (ic_total, mils, civs, ic_prod_list)
        case 'scale':
            scaled_score = ic_total * np.log2((mils/mils_0))
            return (scaled_score, mils, civs, ic_total, ic_prod_list)
        case 'scale_total':
            scaled_score = ic_total * np.log2((mils/mils_0)) * (total/total_0)
            if scaled_score == None:
                scaled_score = 0
            return (scaled_score, mils, civs, ic_total, ic_prod_list)
        case 'scale_civs':
            scaled_score = ic_total * np.log2((mils/mils_0)) * np.log2((civs/civs_0))
            if scaled_score == None:
                scaled_score = 0
            return (scaled_score, mils, civs, ic_total, ic_prod_list)

In [195]:
#production([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
test = [1] * 30 + [0] * 27
production(test, 'scale', 'half year')


(706265.8728971812,
 105.0,
 79.0,
 411998.0863993941,
 [2520.0,
  2573.361,
  2625.6155107351824,
  2676.8300619328743,
  2727.0647488157997,
  2795.9347967607764,
  2863.1084233480597,
  2928.7060369307037,
  2992.8343849559737,
  3513.926931274457,
  3584.6121797608034,
  3653.9035802928897,
  3721.8809619730296,
  3788.616787443473,
  4381.453520530527,
  4477.900836020669,
  4572.270819965095,
  4664.693042234778,
  4755.284089957298,
  4844.1493232442945,
  4931.384336412094,
  5017.076182995574,
  5101.3044096731455,
  5184.141934365595,
  5265.655796321946,
  5345.907800316428,
  5424.955072694562,
  5502.850543594766,
  5579.643366996426,
  5655.379288130662,
  5730.10096610669,
  5878.8133499680325,
  6040.55469911573,
  6932.857417808887,
  7183.7155435575805,
  7453.089813305387,
  7740.783191198528,
  8046.494712500507,
  8369.842851673837,
  8710.386875256105,
  9067.645609323534,
  9441.113368373124,
  9798.82344279986,
  10194.80418338579,
  10605.714519562629,
  11031.

In [165]:
test = [0] * 57
test2 = [1] * 16 + [0] * 41
test3 = [1] * 57

production(test, 'rush', 'half year'), production(test2, 'rush', 'half year'), production(test3, 'rush', 'half year')

#production(test, 'scale'), production(test2, 'scale'), production(test3, 'scale')

#production(test, 'scale_total'), production(test2, 'scale_total'), production(test3, 'scale_total')

#production(test, 'scale_civs'), production(test2, 'scale_civs'), production(test3, 'scale_civs')

((516193.8504638711, 89.0, 45),
 (474992.4152971389, 98.0, 57.0),
 (316115.34711203014, 32, 168.0))

In [179]:
def genetic_algorithm(n_bits, n_pop, n_iter, r_cross, max_target, scenario):

    pop = [np.random.randint(0, 4, n_bits).tolist() for _ in range(n_pop)]
    best, best_eval = 0, production(pop[0], max_target, scenario)
    
    for gen in range(n_iter):
        scores = [production(c, max_target, scenario) for c in pop]
        for i in range(n_pop):
            if scores[i] > best_eval:
                best, best_eval = pop[i], scores[i]
                print("Generation " + str(gen) + ": new best solution found!")
                print(best, best_eval)
                
        #ADD ELITISM PLUS ADDITIONAL COMPLETELY RANDOM SAMPLES HERE
        
        selected = [selection(pop, scores) for _ in range(n_pop)]
        children = []
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i+1]
            c = crossover(p1, p2, r_cross)
            for child in c:
                mutated = mutation(child, 1 / n_bits)
                children.append(mutated)
        #print(np.mean(sum([production(c) for c in pop])))
        #print(np.mean(sum([production(c) for c in children])))
        #print(sum([sum(strat) for strat in pop]))
        pop = children
        
    #SMOOTHING TEST
    best_smoothed = [best[0]]
    for i in range(1, len(best)-1):
        #print((best[i-1],best[i],best[i+1]))
        if best[i-1] == best[i+1]:
            best_smoothed.append(best[i-1])
        else:
            best_smoothed.append(best[i])
    best_smoothed.append(best[-1])
    best_smoothed_eval = production(best_smoothed, max_target, scenario)
    if best_smoothed_eval > best_eval:
        print("Smoothing : new best solution found!")
        print(best_smoothed, best_smoothed_eval)
        return [best_smoothed, best_smoothed_eval]
    
    return [best, best_eval]

In [169]:
genetic_algorithm(10,10, 5, 0.9, 'rush', 'half year') #There should be 66 states of 30 days, ends up on June 5th '41
#Note: states could also be 35 days to reflect the length of the focus tree
#This would be 57 states of 35 days, ending onJune 19th '41 (no leap years)

Generation 0: new best solution found!
[1, 2, 2, 2, 1, 0, 0, 1, 0, 2] (28433.358793712858, 39.0, 42.0)
Generation 0: new best solution found!
[2, 2, 0, 3, 0, 1, 2, 2, 0, 3] (28504.341531800463, 38.0, 40.0)
Generation 1: new best solution found!
[1, 2, 2, 2, 0, 1, 2, 2, 0, 3] (28865.261539123443, 39.0, 39.0)


[[1, 2, 2, 2, 0, 1, 2, 2, 0, 3], (28865.261539123443, 39.0, 39.0)]

In [170]:
#smoothing does not bring better results anymore, at least for 'scale' max_target
#the results for 'scale' seem very similar to those of 'rush'
#It produces 96% of the IC, has 6% more mils and 13% more civs
genetic_algorithm(57,100, 60, 0.9, 'scale', 'half year')

Generation 0: new best solution found!
[0, 0, 2, 2, 2, 1, 2, 3, 1, 3, 0, 3, 2, 0, 2, 2, 1, 3, 1, 2, 0, 0, 0, 2, 2, 1, 3, 0, 3, 2, 0, 3, 2, 0, 0, 0, 3, 1, 1, 2, 1, 2, 3, 1, 2, 1, 3, 3, 3, 0, 2, 0, 1, 3, 2, 3, 2] (359542.9039595827, 61.0, 32.0, 386299.0012854081)
Generation 0: new best solution found!
[0, 0, 2, 3, 2, 2, 2, 0, 0, 0, 0, 3, 1, 2, 0, 0, 1, 3, 2, 0, 1, 3, 0, 2, 3, 1, 1, 2, 2, 2, 2, 0, 1, 3, 2, 0, 3, 1, 3, 2, 3, 0, 1, 1, 3, 1, 3, 1, 2, 2, 2, 1, 0, 1, 2, 3, 1] (387340.34856315516, 63.0, 33.0, 396345.34512484365)
Generation 0: new best solution found!
[1, 0, 0, 1, 1, 0, 0, 2, 3, 3, 3, 0, 2, 0, 0, 2, 1, 2, 2, 3, 2, 1, 0, 0, 1, 1, 2, 1, 2, 3, 1, 2, 3, 0, 3, 1, 1, 0, 0, 2, 2, 2, 3, 3, 0, 0, 2, 2, 3, 1, 0, 2, 0, 3, 2, 0, 0] (418844.03985450766, 66.0, 32.0, 401040.21278077824)
Generation 0: new best solution found!
[0, 0, 1, 1, 2, 0, 2, 1, 0, 1, 0, 1, 0, 1, 2, 1, 1, 3, 0, 2, 0, 3, 3, 3, 0, 0, 3, 2, 0, 0, 2, 0, 0, 1, 3, 3, 1, 0, 1, 0, 0, 3, 3, 3, 2, 2, 1, 1, 2, 2, 0, 2, 0, 3, 1, 3, 2]

Generation 16: new best solution found!
[1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 0, 2, 0, 0, 1, 0, 1, 0, 0, 3, 2, 0, 0, 0, 1, 2, 0, 2, 2, 3, 2, 0, 0, 2, 0] (676880.3241324457, 84.0, 38.0, 486153.7412794426)
Generation 17: new best solution found!
[0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 1, 2, 0] (698152.2539671782, 89.0, 42.0, 473088.3229432477)
Generation 18: new best solution found!
[0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 1, 2, 1, 0, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 1, 2, 0] (702797.9802790035, 87.0, 40.0, 487058.55932745524)
Generation 21: new best solution found!
[0, 0, 0, 0, 2, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 3, 2, 2, 1, 2, 0

[[1,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  0,
  2,
  2,
  2,
  2,
  2,
  0],
 (770926.3717480355, 96.0, 44.0, 486400.3857487259)]

In [201]:
#The best strategy for 'scale_total' produces about 81% percent of the IC that the rush strategy does
#It ends up with 18% extra mils and 66% extra civs though
genetic_algorithm(57,100, 60, 0.9, 'scale_total', 'half year')

Generation 0: new best solution found!
[1, 1, 1, 3, 2, 0, 2, 1, 1, 1, 0, 1, 0, 2, 0, 2, 2, 1, 0, 3, 1, 0, 1, 1, 1, 2, 1, 3, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 2, 2, 1, 2, 0, 1, 1, 1, 2, 2, 1, 3, 2, 0, 3, 1, 0, 1, 3] (696247.6259357209, 69.0, 40.0, 443693.1004783185, [32, 32, 32, 32, 32.0, 32.0, 32.0, 35.0, 35.0, 35.0, 35.0, 37.0, 37.0, 38.0, 40.0, 41.0, 44.0, 46.0, 46.0, 48.0, 47.0, 47.0, 48.0, 48.0, 48.0, 48.0, 50.0, 50.0, 49.0, 50.0, 50.0, 51.0, 52.0, 52.0, 53.0, 55.0, 55.0, 56.0, 56.0, 58.0, 61.0, 61.0, 62.0, 63.0, 63.0, 63.0, 63.0, 65.0, 67.0, 67.0, 67.0, 68.0, 69.0, 68.0, 68.0, 69.0, 69.0])
Generation 0: new best solution found!
[0, 3, 3, 0, 3, 0, 3, 0, 0, 3, 1, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 3, 0, 3, 2, 0, 0, 1, 1, 0, 0, 1, 2, 1, 2, 1, 1, 1, 2, 0, 0, 0, 2, 1, 1, 1, 2, 0, 1, 1, 0, 1, 0, 0, 3, 3] (772829.3828654154, 72.0, 48.0, 423872.3451451605, [32, 32.0, 32.0, 32.0, 32.0, 31.0, 32.0, 31.0, 33.0, 34.0, 33.0, 33.0, 33.0, 35.0, 37.0, 37.0, 38.0, 41.0, 43.0, 44.0, 46.0, 47.0, 47.0, 46.

Generation 11: new best solution found!
[2, 1, 0, 3, 3, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 2, 0, 0, 0, 0, 0, 3, 2] (1124613.2844478248, 86.0, 46.0, 459959.77518558316, [32, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 33.0, 35.0, 35.0, 36.0, 36.0, 36.0, 36.0, 38.0, 40.0, 42.0, 43.0, 45.0, 45.0, 47.0, 47.0, 47.0, 47.0, 47.0, 48.0, 50.0, 50.0, 52.0, 54.0, 56.0, 56.0, 57.0, 59.0, 59.0, 61.0, 61.0, 65.0, 68.0, 68.0, 68.0, 68.0, 71.0, 71.0, 74.0, 76.0, 76.0, 78.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 84.0])
Generation 11: new best solution found!
[1, 3, 0, 0, 3, 0, 3, 0, 0, 3, 1, 3, 0, 0, 1, 0, 3, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 1, 2, 1, 2, 0, 2, 0, 0, 0, 1, 2, 2] (1157613.2270928898, 91.0, 49.0, 422263.9197186609, [32, 32, 32.0, 32.0, 32.0, 32.0, 33.0, 32.0, 34.0, 35.0, 34.0, 34.0, 32.0, 34.0, 36.0, 36.0, 37.0, 36.0, 38.0, 40.0, 42.0, 42.0, 

Generation 24: new best solution found!
[1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 1, 0, 2, 1, 0, 2, 2, 2] (1395610.2004695598, 95.0, 52.0, 465669.52441498794, [32, 32, 32, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 33.0, 35.0, 35.0, 37.0, 37.0, 37.0, 38.0, 40.0, 40.0, 42.0, 43.0, 45.0, 45.0, 47.0, 47.0, 47.0, 47.0, 49.0, 50.0, 50.0, 52.0, 52.0, 54.0, 56.0, 56.0, 58.0, 60.0, 60.0, 62.0, 64.0, 66.0, 68.0, 70.0, 71.0, 73.0, 75.0, 77.0, 77.0, 78.0, 78.0, 81.0, 81.0, 83.0, 86.0, 86.0, 88.0, 90.0, 93.0])
Generation 24: new best solution found!
[1, 1, 1, 1, 3, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 0, 0, 2, 1, 0, 2, 2, 2] (1400216.20326516, 96.0, 52.0, 459626.56036222563, [32, 32, 32, 32, 32, 32.0, 32.0, 32.0, 32.0, 34.0, 35.0, 35.0, 37.0, 37.0, 37.0, 37.0, 39.0, 39.0, 40.0, 42.0, 44.0, 44.0, 46.0, 46.0,

Generation 40: new best solution found!
[1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 2, 2, 0] (1526208.1255705443, 99.0, 63.0, 445218.6234718665, [32, 32, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 33.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 37.0, 39.0, 41.0, 41.0, 43.0, 43.0, 43.0, 43.0, 43.0, 45.0, 45.0, 47.0, 47.0, 49.0, 51.0, 51.0, 54.0, 56.0, 56.0, 58.0, 61.0, 63.0, 65.0, 67.0, 70.0, 72.0, 74.0, 76.0, 76.0, 78.0, 80.0, 80.0, 82.0, 84.0, 86.0, 88.0, 90.0, 93.0, 97.0])
Generation 42: new best solution found!
[1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 2, 2, 0] (1528052.356697502, 99.0, 63.0, 445756.61434608046, [32, 32, 32, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 33.0, 35.0, 35.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 38.0, 38.0, 40.0, 40.0, 42.0,

Generation 59: new best solution found!
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2] (1610198.5284302046, 105.0, 65.0, 425449.6851835635, [32, 32, 32, 32, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 36.0, 36.0, 36.0, 36.0, 36.0, 36.0, 36.0, 36.0, 36.0, 38.0, 38.0, 41.0, 41.0, 43.0, 46.0, 46.0, 49.0, 52.0, 52.0, 54.0, 57.0, 60.0, 62.0, 65.0, 67.0, 70.0, 72.0, 75.0, 75.0, 77.0, 80.0, 82.0, 85.0, 87.0, 89.0, 91.0, 94.0, 98.0, 101.0])
Smoothing : new best solution found!
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2] (1697554.5891683907, 109.0, 66.0, 422424.2963630636, [32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 34.0, 34.0, 37.0, 40.0,

[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  2,
  2],
 (1697554.5891683907,
  109.0,
  66.0,
  422424.2963630636,
  [32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   34.0,
   34.0,
   37.0,
   40.0,
   43.0,
   46.0,
   49.0,
   52.0,
   54.0,
   57.0,
   60.0,
   63.0,
   65.0,
   68.0,
   71.0,
   73.0,
   76.0,
   78.0,
   81.0,
   83.0,
   85.0,
   88.0,
   90.0,
   92.0,
   94.0,
   96.0,
   99.0,
   103.0,
   106.0])]

In [202]:
#'scale_civs' produces 67% of 'rush' IC, but has 4% more mils and 142% more civs
genetic_algorithm(57,100, 60, 0.9, 'scale_civs', 'half year')

Generation 0: new best solution found!
[0, 3, 2, 0, 0, 2, 2, 1, 3, 3, 0, 0, 2, 3, 2, 0, 1, 1, 1, 2, 0, 3, 3, 3, 1, 3, 0, 1, 0, 3, 1, 2, 1, 2, 2, 1, 0, 3, 2, 0, 1, 0, 2, 2, 0, 1, 0, 1, 3, 1, 3, 1, 1, 0, 2, 0, 1] (-76750.3320565012, 63.0, 39.0, 380403.54280316, [32, 32.0, 32.0, 32.0, 32.0, 33.0, 35.0, 37.0, 37.0, 36.0, 35.0, 36.0, 37.0, 39.0, 38.0, 41.0, 43.0, 43.0, 43.0, 43.0, 45.0, 46.0, 45.0, 45.0, 44.0, 44.0, 43.0, 44.0, 44.0, 45.0, 44.0, 44.0, 46.0, 46.0, 49.0, 51.0, 51.0, 52.0, 51.0, 53.0, 54.0, 54.0, 55.0, 57.0, 59.0, 60.0, 60.0, 61.0, 61.0, 60.0, 60.0, 59.0, 59.0, 59.0, 60.0, 62.0, 63.0])
Generation 0: new best solution found!
[1, 0, 2, 2, 1, 0, 0, 2, 2, 2, 1, 3, 2, 1, 3, 1, 2, 3, 1, 1, 2, 2, 3, 3, 0, 2, 2, 1, 1, 0, 1, 1, 3, 0, 2, 3, 0, 2, 1, 3, 3, 1, 3, 1, 3, 3, 0, 0, 0, 1, 0, 0, 3, 0, 2, 1, 3] (-53960.62208562692, 57.0, 40.0, 381269.4521023936, [32, 32, 32.0, 32.0, 33.0, 33.0, 34.0, 35.0, 37.0, 40.0, 42.0, 42.0, 42.0, 44.0, 44.0, 43.0, 43.0, 44.0, 43.0, 43.0, 43.0, 45.0, 47.0, 

Generation 9: new best solution found!
[1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 3, 0, 1, 3, 3, 1, 0, 3, 1, 3, 0, 0, 1, 0, 1, 0, 1, 0, 3, 1, 1, 0, 1, 1, 1, 0, 3, 1, 3, 1, 3, 3, 0, 0, 0, 1, 0, 0, 3, 0, 2, 1, 3] (315214.96523957787, 61.0, 100.0, 293985.59182228235, [32, 32, 32, 32, 32, 32, 32.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0, 33.0, 35.0, 35.0, 33.0, 32.0, 32.0, 34.0, 32.0, 32.0, 30.0, 32.0, 35.0, 35.0, 37.0, 37.0, 40.0, 40.0, 42.0, 40.0, 40.0, 40.0, 43.0, 43.0, 43.0, 43.0, 46.0, 44.0, 44.0, 41.0, 41.0, 38.0, 35.0, 40.0, 44.0, 48.0, 48.0, 52.0, 56.0, 53.0, 57.0, 64.0, 64.0])
Generation 9: new best solution found!
[1, 1, 1, 0, 3, 1, 3, 3, 3, 1, 3, 0, 1, 1, 3, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 3, 1, 0, 1, 0, 0, 1, 0, 0, 0, 3, 2, 1, 1, 3, 1, 3, 1, 3, 3, 0, 0, 0, 1, 0, 0, 3, 1, 1, 0, 1] (326482.1422286875, 63.0, 94.0, 314351.2284571993, [32, 32, 32, 32, 32.0, 32.0, 32.0, 31.0, 30.0, 29.0, 29.0, 27.0, 29.0, 29.0, 29.0, 28.0, 28.0, 28.0, 30.0, 32.0, 34.0, 36.0, 36.0, 36.0, 39.0

Generation 24: new best solution found!
[1, 1, 0, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 3, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1] (535607.1492200937, 78.0, 98.0, 371093.1747996342, [32, 32, 32, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 31.0, 31.0, 31.0, 31.0, 31.0, 31.0, 29.0, 29.0, 29.0, 31.0, 31.0, 33.0, 36.0, 36.0, 38.0, 40.0, 40.0, 42.0, 42.0, 42.0, 42.0, 45.0, 47.0, 47.0, 47.0, 47.0, 47.0, 50.0, 50.0, 50.0, 50.0, 53.0, 51.0, 51.0, 51.0, 54.0, 57.0, 57.0, 61.0, 64.0, 64.0, 64.0, 67.0, 71.0, 74.0, 74.0, 74.0, 78.0])
Generation 25: new best solution found!
[1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 3, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 3, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1] (537334.0095950046, 81.0, 97.0, 361930.3225922366, [32, 32, 32, 32, 32, 32, 32, 31.0, 31.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 28.0, 28.0, 28.0, 30.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0,

Generation 35: new best solution found!
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1] (609987.722506873, 83.0, 100.0, 385081.18197187467, [32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 31.0, 31.0, 31.0, 31.0, 33.0, 35.0, 37.0, 37.0, 37.0, 39.0, 39.0, 39.0, 39.0, 39.0, 39.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 45.0, 45.0, 45.0, 45.0, 48.0, 48.0, 52.0, 55.0, 55.0, 59.0, 59.0, 62.0, 65.0, 69.0, 69.0, 72.0, 76.0, 79.0, 79.0, 79.0, 83.0])
Generation 37: new best solution found!
[1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0] (618308.6318132716, 86.0, 100.0, 376314.98630703695, [32, 32, 32, 32, 32, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 31.0, 31.0, 31.0, 33.0, 35.0, 35.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0

[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 (723297.1945750964,
  98.0,
  106.0,
  362393.29170560144,
  [32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   32,
   36.0,
   40.0,
   44.0,
   44.0,
   49.0,
   49.0,
   53.0,
   57.0,
   62.0,
   66.0,
   70.0,
   74.0,
   78.0,
   82.0,
   86.0,
   90.0,
   94.0])]

In [203]:
genetic_algorithm(57,100, 60, 0.9, 'rush', 'half year')

Generation 0: new best solution found!
[3, 3, 1, 0, 1, 3, 1, 0, 1, 2, 0, 1, 3, 3, 2, 2, 2, 0, 1, 3, 2, 3, 0, 1, 0, 1, 0, 0, 3, 1, 3, 1, 2, 2, 3, 2, 1, 1, 0, 1, 2, 1, 2, 0, 3, 0, 1, 2, 0, 3, 0, 3, 0, 3, 2, 0, 2] (382645.22947461624, 65.0, 37.0, [32, 32.0, 32.0, 32.0, 32.0, 32.0, 31.0, 31.0, 32.0, 32.0, 34.0, 36.0, 36.0, 35.0, 34.0, 37.0, 40.0, 43.0, 44.0, 44.0, 43.0, 45.0, 44.0, 45.0, 45.0, 46.0, 46.0, 48.0, 49.0, 48.0, 48.0, 47.0, 47.0, 49.0, 51.0, 50.0, 53.0, 53.0, 53.0, 54.0, 54.0, 56.0, 56.0, 58.0, 59.0, 58.0, 59.0, 59.0, 60.0, 61.0, 61.0, 61.0, 60.0, 61.0, 60.0, 62.0, 63.0])
Generation 0: new best solution found!
[1, 1, 3, 0, 1, 0, 1, 1, 3, 2, 2, 3, 2, 3, 3, 3, 0, 0, 1, 0, 2, 2, 0, 2, 0, 3, 0, 1, 3, 0, 1, 2, 1, 2, 1, 0, 2, 0, 0, 1, 3, 1, 1, 0, 2, 0, 3, 1, 3, 0, 2, 0, 1, 2, 2, 2, 0] (390736.7735312829, 68.0, 35.0, [32, 32, 32, 32.0, 32.0, 32.0, 33.0, 33.0, 33.0, 32.0, 34.0, 37.0, 36.0, 39.0, 38.0, 37.0, 36.0, 37.0, 39.0, 39.0, 40.0, 42.0, 44.0, 45.0, 47.0, 48.0, 47.0, 48.0, 48.0, 47

Generation 11: new best solution found!
[0, 2, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 2, 0, 2, 0, 0, 0, 2, 0, 2, 1, 1, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 2, 0, 2, 2, 0, 1, 1, 2, 1, 2, 1, 0] (481109.8897384194, 75.0, 36.0, [32, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 34.0, 35.0, 35.0, 38.0, 40.0, 43.0, 44.0, 45.0, 46.0, 48.0, 50.0, 52.0, 52.0, 52.0, 53.0, 54.0, 55.0, 57.0, 58.0, 58.0, 58.0, 58.0, 59.0, 60.0, 60.0, 62.0, 63.0, 64.0, 65.0, 65.0, 65.0, 65.0, 65.0, 65.0, 65.0, 65.0, 66.0, 67.0, 69.0, 71.0, 72.0, 72.0, 72.0, 73.0, 73.0, 74.0, 74.0])
Generation 11: new best solution found!
[0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 2, 0, 2, 0, 0, 0, 2, 0, 2, 1, 1, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 0, 1, 0, 0, 1, 1, 0, 3, 1, 1] (482677.39756536356, 73.0, 41.0, [32, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 34.0, 35.0, 35.0, 38.0, 40.0, 42.0, 43.0, 44.0, 46.0, 49.0, 50.0, 52.0, 52.0, 52.0, 53.0, 54.0, 55.0, 57.0, 58

Generation 23: new best solution found!
[0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 2, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 2, 0, 1, 0, 0, 1, 1, 0, 2, 1, 2] (490905.60367938277, 79.0, 41.0, [32, 32.0, 32.0, 32.0, 32.0, 32.0, 33.0, 33.0, 34.0, 36.0, 37.0, 39.0, 39.0, 41.0, 42.0, 45.0, 46.0, 47.0, 49.0, 50.0, 51.0, 51.0, 52.0, 52.0, 53.0, 54.0, 54.0, 56.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 63.0, 65.0, 66.0, 66.0, 67.0, 67.0, 68.0, 68.0, 68.0, 69.0, 69.0, 69.0, 70.0, 72.0, 73.0, 73.0, 74.0, 75.0, 75.0, 75.0, 76.0, 77.0, 77.0])
Generation 23: new best solution found!
[0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 2, 1, 0, 1, 1, 0, 2, 0, 1, 0, 0, 1, 1, 1, 2, 0, 0] (491018.12290579686, 77.0, 40.0, [32, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 33.0, 35.0, 36.0, 38.0, 38.0, 40.0, 41.0, 44.0, 46.0, 47.0, 48.0, 49.0, 50.0, 52.0, 53.0, 53.0, 55.0, 56.0, 57.0, 59.0, 5

Generation 38: new best solution found!
[0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 1, 1, 0, 1, 1, 0, 2, 0, 2, 0, 0, 1, 1, 2, 1, 1, 2] (501247.2498663747, 80.0, 38.0, [32, 32.0, 32.0, 32.0, 32.0, 32.0, 33.0, 33.0, 34.0, 36.0, 37.0, 39.0, 40.0, 42.0, 43.0, 45.0, 46.0, 47.0, 49.0, 50.0, 51.0, 54.0, 55.0, 56.0, 58.0, 59.0, 60.0, 61.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 66.0, 67.0, 68.0, 69.0, 71.0, 71.0, 72.0, 72.0, 72.0, 73.0, 73.0, 73.0, 74.0, 75.0, 75.0, 77.0, 78.0, 78.0, 78.0, 78.0, 79.0, 79.0, 79.0])
Generation 38: new best solution found!
[0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 0, 2, 0, 1, 0, 0, 1, 1, 1, 2, 1, 0] (501482.29121718224, 82.0, 43.0, [32, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 34.0, 35.0, 37.0, 38.0, 39.0, 39.0, 41.0, 43.0, 44.0, 45.0, 47.0, 48.0, 50.0, 51.0, 53.0, 54.0, 55.0, 57.0, 58.0, 59.0, 61.0, 62

[[0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  2,
  2,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  2,
  0,
  0],
 (511333.8602396654,
  87.0,
  44.0,
  [32,
   32.0,
   32.0,
   32.0,
   32.0,
   32.0,
   33.0,
   34.0,
   36.0,
   37.0,
   38.0,
   40.0,
   41.0,
   43.0,
   44.0,
   45.0,
   47.0,
   48.0,
   50.0,
   51.0,
   52.0,
   53.0,
   55.0,
   56.0,
   57.0,
   58.0,
   60.0,
   61.0,
   62.0,
   63.0,
   64.0,
   65.0,
   66.0,
   67.0,
   68.0,
   70.0,
   71.0,
   72.0,
   73.0,
   74.0,
   75.0,
   76.0,
   77.0,
   78.0,
   79.0,
   79.0,
   80.0,
   82.0,
   82.0,
   83.0,
   84.0,
   85.0,
   85.0,
   85.0,
   85.0,
   86.0,
   86.0])]

In [193]:
#The delay in research does not cause that big of a production loss
#Especially interesting is that going one year ahead is only marginally better
#than half a year, good to know in actual games

test = [0] * 57

(production(test, 'rush', 'one year'), production(test, 'rush', 'half year'), \
 production(test, 'rush', 'current'), production(test, 'rush', 'test'))

((525609.0511625487, 89.0, 45),
 (516193.8504638711, 89.0, 45),
 (498766.930269018, 89.0, 45),
 (283782.58950169286, 79.0, 45))